In [1]:
#토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sarcasm_0526")

LangSmith 추적을 시작합니다.
[프로젝트명]
sarcasm_0526


In [3]:
#llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o-mini")

In [4]:
# prompt 가져오기
from langchain_core.prompts import load_prompt

prompt = load_prompt(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/generate_context.yaml",
    encoding="utf-8",
)
prompt

PromptTemplate(input_variables=['format_instructions', 'speaker_type', 'subject', 'text'], input_types={}, partial_variables={}, template='다음은 사람들 간의 대화 내용입니다.\n주제:{subject}\n화자 유형:{speaker_type}\n대화 내용:{text}\n\n이 대화를 읽고, 이 대화가 어떤 상황(주제, 사건)에 관한 것인지 한 문장으로 요약해주세요.(context)\n또한, 해당 상황이 긍정인지 부정인지 분류해주세요.(sentiment)\n출력은 \'context\'와 \'sentiment\'가 반드시 포함되어야 합니다.\n\n예시:\n입력:\n주제: 음식\n화자 유형: 다자간 대화\n대화 내용:\n1 : 어제 새로 생긴 파스타 집 가봤어요?  \n2 : 아니요, 어떤가요?  \n1 : 별로였어요. 해산물 파스타는 비리고, 면도 덜 익었더라고요.  \n3 : 분위기는 괜찮다던데요?  \n1 : 분위기는 괜찮았지만, 서비스도 느리고 전반적으로 실망스러웠어요.\n\n출력:\n{{"context": "새로 오픈한 파스타 가게의 음식 맛과 서비스가 기대에 못 미쳐 실망한 대화",\n  "sentiment": "부정"}}\n\n{format_instructions}\n\n이제 다음 대화를 주제를 요약하세요:\n\n입력 대화:\n{text}\n\n출력:\n')

In [5]:
# ① 출력 데이터 모델 정의 (score, reason)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional

class SentimentResult(BaseModel):
    context: str = Field(description="대화 상황을 한 문장으로 요약")
    sentiment: str = Field(description="상황의 긍정/부정 분류")

output_parser = PydanticOutputParser(pydantic_object=SentimentResult)

In [6]:
format_instructions = output_parser.get_format_instructions()
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
# 체인 생성
chain = prompt | llm | output_parser

In [32]:
# 데이터셋 1000개 만들기
import os
import shutil

source_folder = "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/TL_05. NATEON"
target_folder = "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/sixth_data_0526"
os.makedirs(target_folder, exist_ok=True)

# JSON 파일 목록
json_files = [f for f in os.listdir(source_folder) if f.endswith(".json")]

# 앞 1000개만 복사
for file in json_files[500:600]:
    shutil.copy(os.path.join(source_folder, file), os.path.join(target_folder, file))

print("앞 100개 JSON 파일을 복사했습니다.")

앞 100개 JSON 파일을 복사했습니다.


In [33]:
# 파일 안에 json 파일 개수 파악
import os

folder_path = "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/sixth_data_0526"

json_count = sum(
    1 for file in os.listdir(folder_path)
    if file.endswith(".json")
)

print(f"JSON 파일 개수: {json_count}개")

JSON 파일 개수: 500개


In [34]:
#json 파일 불러오기
import os
import json

entries = []

# 폴더 안의 모든 JSON 파일 처리
json_folder_path = '/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/sixth_data_0526'

for filename in os.listdir(json_folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(json_folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f) 

        # info 추출
        for item in data['info']:
            annotations = item['annotations']
            subject = annotations.get('subject', '')
            speaker_type = annotations.get('speaker_type', '')
            text = annotations.get('text', '')

            # dict 형태로 저장
            entry = {
                "subject": subject,
                "speaker_type": speaker_type,
                "text": text
            }
            entries.append(entry)

In [35]:
# 결과를 저장할 리스트
results = []

for entry in entries:
    # 체인 호출
    parsed = chain.invoke({
        "subject": entry["subject"],
        "speaker_type": entry["speaker_type"],
        "text": entry["text"]
    })
    
    # 결과 저장
    results.append({
        "subject": entry["subject"],
        "speaker_type": entry["speaker_type"],
        "context": parsed.context,
        "sentiment": parsed.sentiment
    })

In [37]:
test_df_1 = test_df[test_df["sentiment"] == '부정']
test_df_1.head()

,subject,speaker_type,context,sentiment
1,사회이슈,다자간 대화,방송인 송선미씨 남편 피살 사건과 관련된 충격적인 소식과 진돗개 물림 사건에 대한 ...,부정
4,주거와 생활,1:1,집에서 발생하는 물 새는 문제와 그로 인한 스트레스에 대한 대화,부정
8,건강,1:1,대학 병원에 다니는 친구의 난치병과 보험에 대한 대화,부정
9,건강,다자간 대화,부스터샷 접종에 대한 우려와 경험을 나누는 대화,부정
11,식음료,1:1,해외 여행 중의 기억과 전 남자친구와의 갈등에 대한 대화,부정


In [38]:
test_df_1.to_csv("/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/negative_data_6_0531.csv", encoding = "utf-8-sig")

In [21]:
test_df["sentiment"].value_counts()

sentiment
긍정    266
부정    163
중립     71
Name: count, dtype: int64